In [ ]:
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, recall_score, precision_score
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import IsolationForest
import numpy as np

from datasetStuff import expandDataset

In [ ]:
shutil.rmtree('./model_checkpoints', ignore_errors=True)

X = df_final
y = labels

input_shape = 600
pca_test = PCA(n_components=input_shape)

random_state = 42

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.235, random_state=random_state, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, test_size=0.5, random_state=random_state, stratify=y_val)


scaler = StandardScaler()
X_train_scaled, X_val_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_val), scaler.transform(X_test)

X_train_scaled, X_val_scaled, X_test_scaled = pca_test.fit_transform(X_train_scaled), pca_test.transform(X_val_scaled), pca_test.transform(X_test_scaled)


detector = IsolationForest(contamination = 0.1).fit(X_val_scaled)
pred_anomalies = detector.predict(X_val_scaled)
X_val_scaled = X_val_scaled[pred_anomalies == 1]
y_val = y_val[pred_anomalies == 1]

oversample = SMOTE()
X_val_scaled, y_val = oversample.fit_resample(X_val_scaled, y_val)

zero_weight = (np.sum(y_train==1)/np.sum(y_train==0))*1.7

#Create a simple neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='gelu',
              input_shape=(input_shape,),
              kernel_regularizer=tf.keras.regularizers.L1(0.0012),
              activity_regularizer=tf.keras.regularizers.L2(0.0003),
              kernel_initializer=tf.keras.initializers.GlorotUniform()
              ),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Dense(70, activation='gelu',
              kernel_regularizer=tf.keras.regularizers.L1(0.0012),
              activity_regularizer=tf.keras.regularizers.L2(0.0003),
              kernel_initializer=tf.keras.initializers.GlorotUniform()
              ),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Dense(2, activation='sigmoid',
              kernel_initializer=tf.keras.initializers.GlorotUniform(),
                          ),
])

# Convert target labels to one-hot encoding
y_train_categorized, y_val_categorized = to_categorical(y_train, num_classes=2), to_categorical(y_val, num_classes=2)

# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# Define a callback to save checkpoints
checkpoint_filepath = "model_checkpoints/checkpoint_{epoch:02d}.h5"

# Save a model checkpoint every 100 epochs
model_save = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    period=2,  # Save every 100 epochs
)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.AUC()])
epochs = 200

# Train the model
history = model.fit(X_train_scaled, y_train_categorized, epochs=epochs, batch_size=32, class_weight={0:1, 1:1}, validation_data=(X_val_scaled, y_val_categorized), callbacks=[early_stopping, model_save])